In [ ]:
%env TF_KERAS=1

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

import codecs
import unicodedata
import re
import os
import numpy as np

import io
import time

# import MeCab

In [3]:
# tf.enable_eager_execution()
# tf.compat.v1.disable_eager_execution() #tf2 code

print(tf.__version__, tf.keras.__version__)

2.0.0-rc0 2.2.4-tf


In [4]:
# jpn-english material
path_to_file = "./jpn-eng/jpn.txt"

In [5]:
# bert model parameters
SEQ_LEN = 64
BATCH_SIZE = 64
OUTPUT_LAYER_NUM = 4
LEARNING_RATE = 1e-5

PRETRAINED_PATH = 'bert-master\\multi_cased_L-12_H-768_A-12'
CONFIG_PATH = os.path.join(PRETRAINED_PATH, 'bert_config.json')
CHECKPOINT_PATH = os.path.join(PRETRAINED_PATH, 'bert_model.ckpt')
VOCAB_PATH = os.path.join(PRETRAINED_PATH, 'vocab.txt')

In [6]:
# load bert model
bert_base = load_trained_model_from_checkpoint(
  CONFIG_PATH,
  CHECKPOINT_PATH,
  training=False,
  trainable=False,
  output_layer_num=OUTPUT_LAYER_NUM,
  seq_len=SEQ_LEN
)

In [7]:
# extract = K.function([bert_base.layers[0].input, bert_base.layers[1].input], [bert_base.layers[-1].output])

In [8]:
# prepare token->idx dictionary
def make_token_dict(vocab_path):
  token_dict = {}
  with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
      if line != ' \n':
        token = line.strip()
      else:
        token = line.strip('\n')
      token_dict[token] = len(token_dict)
  return token_dict

In [9]:
def preprocess_eng_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '[CLS] ' + w + ' [SEP]'
    return w

def preprocess_jpn_sentence(w):
    m = MeCab.Tagger ("-Owakati")
    w = '[CLS] ' + m.parse(w).strip().strip('\n') + ' [SEP]'
    return w

In [10]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    eng, jpn = [], []
    for l in lines[:num_examples]:
#         (eng_sentence, jpn_sentence) = l.split('\t')
        (eng_sentence, _) = l.split('\t')
        eng.append(preprocess_eng_sentence(eng_sentence))
#         jpn.append(preprocess_jpn_sentence(jpn_sentence))
    return eng, jpn

In [11]:
eng, jpn = create_dataset(path_to_file, None)

In [12]:
print(eng[3])
# print(jpn[3])

[CLS] hi . [SEP]


In [13]:
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
    # convert words(of a sentence) into word indexes
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
                     filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)
    # padding word indexes(of sentences) to the same length(using maximum length of all sentences)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post')

    return tensor, lang_tokenizer

# def load_dataset(path, num_examples=None):
#     # creating cleaned input, output pairs
#     targ_lang, inp_lang = create_dataset(path, num_examples)

#     input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
#     target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

#     return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [14]:
def load_dataset(path, input_vocab_path, num_examples=None, verbose=False):
  # creating cleaned input, output pairs
  targ_lang, _ = create_dataset(path, num_examples)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  input_token_dict = make_token_dict(input_vocab_path)
  inp_lang_tokenizer = Tokenizer(input_token_dict, cased=True)
  
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  inp_ids, inp_segments = [], []
  for l in lines[:num_examples]:
      (_, jpn_sentence) = l.split('\t')
      id, segment = inp_lang_tokenizer.encode(jpn_sentence, max_len=SEQ_LEN)
      inp_ids.append(id)
      inp_segments.append(segment)
      if verbose:
        print('{}->{}{}'.format(jpn_sentence, id, segment))

  input_tensor = []
  for i, s in zip(inp_ids, inp_segments):
    input_tensor.append([i,s])        

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [15]:
# inp_ids, inp_segments, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, VOCAB_PATH, 3)

In [16]:
# Try experimenting with the size of that dataset
num_examples = 45000
# input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, VOCAB_PATH, num_examples)


# Calculate max_length of the target tensors
# max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
max_length_targ, max_length_inp = max_length(target_tensor), SEQ_LEN

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

36000 36000 9000 9000


In [17]:
print(max_length_targ, max_length_inp)

29 64


In [ ]:
# def convert(lang, tensor):
#   if isinstance(lang, tf.keras.preprocessing.text.Tokenizer):
#     for t in tensor:
#       if t!=0:
#         print ("%d ----> %s" % (t, lang.index_word[t]))
#   else:
#     print (lang.decode(tensor))

# print ("Input Language; index to word mapping")
# convert(inp_lang, input_tensor_train[0][0])
# print ()
# print ("Target Language; index to word mapping")
# convert(targ_lang, target_tensor_train[0])

In [18]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 3072 # concatenation of last 4 layer ouputs of Transformer
units = 1024
# vocab_inp_size = len(inp_lang.word_index)+1
vocab_inp_size = len(inp_lang._token_dict)
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [19]:
class Encoder(tf.keras.Model):
    def __init__(self, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [20]:
# input_tensor_train = np.array(input_tensor_train)
# target_tensor_train = np.array(target_tensor_train)

encoder = Encoder(units, BATCH_SIZE)
# sample input
sample_hidden = encoder.initialize_hidden_state()
example_input_batch = input_tensor_train[:64]

sample_bert_output = bert_base.predict([example_input_batch[:, 0, :], example_input_batch[:, 1, :]], batch_size=BATCH_SIZE)

In [ ]:
# print(input_tensor_train[:64].shape)
# print(example_input_batch[:, 0, :].shape)
# print(np.array(sample_bert_output).shape)
# print(input_tensor_train[:2])

In [21]:
sample_output, sample_hidden = encoder(np.array(sample_bert_output), sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 64, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_with_bert_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt-45k-30epoch")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
# @tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
#         prepared_inp = tf.concat([tf.expand_dims(inp[:, 0, :], 0), tf.expand_dims(inp[:, 1, :], 0)], 0)
#         prepared_inp = tf.stack([inp[:, 0, :], inp[:, 1, :]])
#         print(prepared_inp.shape)
        bert_embedding = bert_base.predict([inp[:, 0, :], inp[:, 1, :]], batch_size=BATCH_SIZE)
#         bert_embedding = bert_base.predict({'Input-Token':prepared_inp[0],'Input-Segment':prepared_inp[1]}, batch_size=BATCH_SIZE)
#         bert_embedding = bert_base.predict([prepared_inp[0],prepared_inp[1]], batch_size=BATCH_SIZE)

        
#         print(bert_embedding.shape)        
        
        enc_output, enc_hidden = encoder(bert_embedding, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['[cls]']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
# print(type(input_tensor_train), input_tensor_train.shape)
# print(type(dataset), tf.compat.v1.data.get_output_types(dataset), tf.compat.v1.data.get_output_shapes(dataset))
# iterator = dataset.make_one_shot_iterator()
# next_x, next_y = iterator.get_next()
# print(type(next_x), next_x.shape)
# print(type(next_y), next_y.shape)

In [ ]:
# prepared_x = tf.stack([next_x[:, 0, :], next_x[:, 1, :]])
# print(prepared_x.shape)
# embedding_x = bert_base.predict([prepared_x[0],prepared_x[1]], batch_size=None, steps=1)
# print(embedding_x.shape)

In [ ]:
# enc_hidden = encoder.initialize_hidden_state()
# enc_output, enc_hidden = encoder(embedding_x, enc_hidden)
# dec_hidden = enc_hidden
# dec_input = tf.expand_dims([targ_lang.word_index['[cls]']] * BATCH_SIZE, 1)
# loss = 0
# for t in range(1, next_y.shape[1]):
#   # passing enc_output to the decoder
#   predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

#   loss += loss_function(next_y[:, t], predictions)

#   # using teacher forcing
#   dec_input = tf.expand_dims(next_y[:, t], 1)
# batch_loss = (loss / int(next_y.shape[1]))

In [ ]:
# tf.trainable_variables()
# with tf.Session() as sess:
#   tf.global_variables_initializer()
#   enc_hidden = encoder.initialize_hidden_state()
#   total_loss = 0
#   while True:
#     try:
#       next_x, next_y = iterator.get_next()
#       prepared_x = tf.stack([next_x[:, 0, :], next_x[:, 1, :]])
#       bert_embedding = bert_base.predict([prepared_x[0],prepared_x[1]], batch_size=None, steps=1)
#       enc_output, enc_hidden = encoder(bert_embedding, enc_hidden)
#       dec_hidden = enc_hidden
#       dec_input = tf.expand_dims([targ_lang.word_index['[cls]']] * BATCH_SIZE, 1)
#       for t in range(1, next_y.shape[1]):
#         # passing enc_output to the decoder
#         predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

#         loss += loss_function(next_y[:, t], predictions)

#         # using teacher forcing
#         dec_input = tf.expand_dims(next_y[:, t], 1)
#       batch_loss = (loss / int(next_y.shape[1]))
#       sess.run(batch_loss)
#     except tf.errors.OutOfRangeError:
#       print("End of dataset")  # ==> "End of dataset"


In [ ]:
# EPOCHS = 4

# for epoch in range(EPOCHS):

#     start = time.time()

#     enc_hidden = encoder.initialize_hidden_state()
#     total_loss = 0
    
#     iterates = len(input_tensor_train) // BATCH_SIZE
#     for batch in range(iterates):
#         print(input_tensor_train[batch * 64:(batch+1) * 64].shape, type(input_tensor_train[batch * 64:(batch+1) * 64]))
#         inp = input_tensor_train[batch * 64:(batch+1) * 64]
#         targ = target_tensor_train[batch * 64:(batch+1) * 64]
#         batch_loss = train_step(inp, targ, enc_hidden)
#         total_loss += batch_loss        
# #     for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
# #         batch_loss = train_step(inp, targ, enc_hidden)
# #         total_loss += batch_loss

#         if batch % 100 == 0:
#             print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
#                                                          batch,
#                                                          batch_loss.eval(session=tf.Session())))
#     # saving (checkpoint) the model every 2 epochs
#     if (epoch + 1) % 2 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)

#     print('Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                         total_loss / steps_per_epoch))
#     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
EPOCHS = 30

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
#         print(inp.shape, targ.shape)
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
def evaluate(sentence, verbose=False):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    if verbose == True:
        print('Original sentence: {}'.format(sentence))

    sentence = preprocess_jpn_sentence(sentence)
    if verbose == True:
        print('Preprocessed sentence: {}'.format(sentence))

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    if verbose == True:
        print('Word sequences of sentence: {}'.format(inputs))
        
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    if verbose == True:
        print('Paded sequences of sentence: {}'.format(inputs))

    inputs = tf.convert_to_tensor(inputs)
    if verbose == True:
        print('Tensor of sentence: {}'.format(inputs))

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    if verbose == True:
        print('enc_out: {}'.format(enc_out))
        print('enc_hidden: {}'.format(enc_hidden))

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    if verbose == True:
        print('dec_input: {}'.format(dec_input))

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
        if verbose == True:
            print('dec_input: {}'.format(dec_input))

    return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
def translate(sentence, verbose=False):
    result, sentence, attention_plot = evaluate(sentence, verbose)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate('雨が嫌い。')

In [ ]:
translate('いい天気ですね。', verbose=True)